## Node embedding

Source: https://towardsdatascience.com/node-embeddings-for-beginners-554ab1625d98

Machine Learning algos **need some sort of vector input to be executed**. 
Since a network consists of connected nodes (with attributes), we need to use **node embedding to represent nodes as vectros**.

Approaches for node embedding can also work for **graph or edge embedding**.

### Classical ML approaches

#### Linear regression

Learn slope and intercept that model the linear relationship between the **ordered feature vector** and the class.

![image](images/linear_regression.png)

#### CNN

Used for image classification. An image is a an **ordered matrix with pixel values (features)**. Layers of convolution (image processing) extract features from the image and a fully connected layer generates the final output.

![image](images/cnn.png)

#### RNN

The input data are, for example, measurements from a time series, thus odered data.

### Issue with graphs and node embeddings

Unlike input data for classical Machine Learning, graphs do not have ordering (first/last nodes, top/bottom, left/right).

Still, we need a procedure that returns a vector, either for a graph, a node or an edge. 

Finding a vector for a node is possible with node representation learning. The result of such representation learning is a node embedding (and so does graph embedding, edge embedding exist). 

*Embeddings should capture the graph topology, relationships between nodes and further relevant information*. **How the embeddings should capture this inherent information of the graph is not fixed. It depends on the questions we ask about the network.**

![image](images/embedding_concept.png)

Examples of node embedding are:
1. node2vec
2. deepwalk

Node embedding must be learned (**unsupervised**) during the training. The initialization of the *d*-space of the embedded nodes is random.
1. decide how large is the embedding space
2. randomly initialize embeddings for each node/graph/edge
3. learning the embeddings by repeatedly incrementally improve the embeddings such that it reflects the similarity in the network (loop)

**Once embedding is known, we apply standard ML algos**.

### GraphSAGE

Source: https://medium.com/analytics-vidhya/ohmygraphs-graphsage-and-inductive-representation-learning-ea26d2835331

Source: https://medium.com/analytics-vidhya/ohmygraphs-graphsage-in-pyg-598b5ec77e7b

GraphSAGE is an iterative algorithm that learns graph embeddings for every node in a certain graph. The novelty of GraphSAGE is that it was the first work to create inductive node embeddings in an unsupervised manner! (Prior to GraphSAGE, most node embedding models were based on spectral decomposition/matrix factorization methods.)

**goal**: The goal of GraphSAGE is to learn a representation for every node based on some combination of its neighbouring nodes, parametrized by **h**.

Let:

- **x**_v: feature vector of the node *v*.
- *k*: depth (how many layers of neighbor nodes are considered to update each node *v*)
- **h**^k_v: embedded representation of the node *v* at the *k*-th iteration.
- **z**_v: final embedding of the node *v*.

![image](images/sage.png)

Given a target node *v*, a layer works as follow:
1. **node embedding** of all the nodes *u* connected to the target node *v*. This results into a list of embedded nodes **h**^k_u
2. **aggregate** all the embedded nodes **h**^k_u into the node representation (embedded) **a**^k_v. The aggregate function must be differentiable, and it could be as simples as an averaging function or as complex as a neural network.
3. **update** the node representation **h**^(k+1)_v using the aggregated representation **a**^k_v and the embedded neighbors **h**^k_u. The update function must be differentiable, and it could be as simples as an averaging function or as complex as a neural network.


## graphSAGE 

Sample implementation with PyTorch:
- small graph: PyTorchGeometry
- large graph: Deep Graph Library


**Graph convolution**: change the feature space of every node in the graph, but the graph structure does not change. In the picture, the convolution "transforms" the node features from a 4d vector to a 3d vector.

![image](images/graph_convolution.png)


The GraphSAGE model is simply a bunch of stacked SAGEConv layers on top of each other. The below model has 3 layers of convolutions.



In [12]:
from torch_geometric.nn import SAGEConv
import torch


class GraphSAGE(torch.nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        """
        In the __init__, we list the functions of the layers that will be used to create the NN in the forward."""
        super().__init__()
        
        # Dropout: machine learning technique where you remove (or "drop out") 
        # units in a neural net to simulate training large numbers of architectures simultaneously. 
        # Importantly, dropout can drastically reduce the chance of overfitting during training.
        self.dropout = dropout
        
        # define 3 layers of convolution of graghSAGE
        # these are only functions to be learned in the training and shall be saved as internal variable
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.conv3 = SAGEConv(hidden_dim, out_dim)
    
    def forward(self, data):
        """
        Forward propagation: the calculation process as the output layers values from the input data.
        In the forward method, we create the actual NN to be trained.
        """
        
        x = self.conv1(data.x, data.adj_t)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)
        
        x = self.conv2(x, data.adj_t)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)
        
        x = self.conv3(x, data.adj_t)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)
        return torch.log_softmax(x, dim=-1)
    
    def train(model, data, train_idx, optimizer):
        model.train()
        optimizer.zero_grad()
        out = model(data)[train_idx]
        loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
        loss.backward()
        optimizer.step()
        return loss.item()
    @torch.no_grad()
    def test(model, data, split_idx, evaluator):
        model.eval()
        out = model(data)
        y_pred = out.argmax(dim=-1, keepdim=True)
        train_acc = evaluator.eval({
            'y_true': data.y[split_idx['train']],
            'y_pred': y_pred[split_idx['train']],
        })['acc']
        valid_acc = evaluator.eval({
            'y_true': data.y[split_idx['valid']],
            'y_pred': y_pred[split_idx['valid']],
        })['acc']
        test_acc = evaluator.eval({
            'y_true': data.y[split_idx['test']],
            'y_pred': y_pred[split_idx['test']],
        })['acc']
        return train_acc, valid_acc, test_acc

In [8]:
import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset

device = f'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
dataset = PygNodePropPredDataset(name='ogbn-products',
                                 transform=T.ToSparseTensor())
data = dataset[0]
# this dataset comes with train-val-test splits predefined for benchmarking
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

In [9]:
print(f' dataset has {data.num_nodes} nodes where each node has a {data.num_node_features} dim feature vector')
print(f' dataset has {data.num_edges} edges where each edge has a {data.num_edge_features} dim feature vector')
print(f' dataset has {dataset.num_classes} classes')
print(split_idx['train'].shape)
print(split_idx['valid'].shape)
print(split_idx['test'].shape)

 dataset has 2449029 nodes where each node has a 100 dim feature vector
 dataset has 123718280 edges where each edge has a 0 dim feature vector
 dataset has 47 classes
torch.Size([196615])
torch.Size([39323])
torch.Size([2213091])


since we need to train on a certain set of nodes and validate/test on another set of nodes, we just mask out the gradients we want with the indexes of the nodes in our train/val/test set!

In [10]:
# compute activations for train subset
out = model(data)[train_idx]
# get gradients for train subset
loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
# evaluate model on test set
out = model(data)[test_idx]

NameError: name 'model' is not defined